In [1]:
#read all files into an array (dataframes)
import numpy as np
import os
import pandas as pd
from IPython.display import display
import datetime
import re
import json
directory = os.getcwd() 

def read_data_from_files(directory):
  dataframes= []
  for root,dirs,files in os.walk(directory):
      for file in files:
         if (file.endswith("Buildingslogs.csv") & os.path.exists(file)):
            col_names = pd.read_csv(file, nrows=0).columns

            dataframes.append(pd.read_csv(file,sep=";",low_memory=False,header=[0],skiprows=[1]))
  return dataframes


#group the data by building where d[str(serial)] = group

def group_data_by_building(data):
  d = {}
  for data in data :
    for serial, group in data.groupby('serial'):
      d[str(serial)] = group
  return d


def int_to_time(input,hours_mapping):
  return datetime.datetime(2021, 1, 1) + datetime.timedelta(hours=input/hours_mapping)

def hour_rounder(t):
    # Rounds to nearest hour by adding a timedelta hour if minute >= 30
    return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour)
               +datetime.timedelta(hours=t.minute//30))  

def map_to_real_time(data):
  # hou much is an hour in game time 
  # for example 1hour = 2000 game time unite
  res=data
  hour_in_numbers = (int(data[0]["time"].max()) - int(data[0]["time"].min())) / (365*24)
  for df in res :
    df["time"]=df["time"].apply(int_to_time,args=[hour_in_numbers])
    #df["time"]=df["time"].apply(hour_rounder)
    df.time=df.time.round("0.1S")
  return res  

def read_orders_data_from_files(directory):
  dataframes= []
  for root,dirs,files in os.walk(directory):
      for file in files:
         if (file.endswith("orderslogs.csv") & os.path.exists(file)):
            col_names = pd.read_csv(file, nrows=0).columns

            dataframes.append(pd.read_csv(file,sep=";",low_memory=False,header=[0],skiprows=[1]))
  return dataframes

#get one buildings stockes according to IBP Stock
#data_line is one line where wares != nan of the original data
def data_line_to_one_building_stock_IBP(position):
  #locations = pd.DataFrame(columns=['LOCNO','LOCTYPE','NAME1','NAME2','LAND1','REGIO','ORT01','PSTLZ','PSTL2','STRAS','LOEVM','TIME_ZONE','TELF1','FABKL','KUKLA','NODETYPE','CUST_ATTR1','CUST_ATTR2','CUST_ATTR3','CUST_ATTR4','CUST_ATTR5','CUST_ATTR6','CUST_ATTR7','CUST_ATTR8','CUST_ATTR9','CUST_ATTR10','LONGITUDE','LATITUDE'])
  #stocks = pd.DataFrame(columns=['MATNR','LOC_WERKS','VRFKZ','MNG01'])
  locatins=["0"]*4
  locatins[0]=position
  locatins[1]="V"
  locatins[2]="CET"
  locatins[3]="03"
  #locations["LOCNO"]=position
  #locations["LOCTYPE"]="V"
  #locations["TIME_ZONE"]="CET"
  #locations["FABKL"]="03"
  return locatins
  #group_data_by_building

def write_series_of_location_to_csv_as_IBPlocation(locationseries_nameseries,filename):

  IBPlocations = pd.DataFrame(columns=['MANDT','LOGSYS','LOCID','LOCATIONTYPE','PRODUCTIONCALID','GEOLATITUDE','GEOLONGITUDE','LOCATIONDEL','LOCCITY','LOCCOUNTRY','LOCNAME','LOCNAME2','LOCATIONREGION','LOCTIMEZONE','PHONE','PLANTTYPE','POBOXZIP','STREET','ZIP'])
  IBPlocations[["LOCID","LOCNAME"]]=locationseries_nameseries
  
  #x = set([x[0] for x in test["position"]])
  #locations["'LATITUDE'"]=list(x)
  IBPlocations["MANDT"]="001"
  IBPlocations["LOGSYS"]="WDL"
  IBPlocations["LOCTYPE"]="P"
  IBPlocations["LOCTIMEZONE"]="CET"
  IBPlocations["PRODUCTIONCALID"]="03"
  #IBPlocations.drop_duplicates(subset ="LOCNO",keep = False, inplace = True)
  IBPlocations.to_csv("locations"+filename+ ".csv",sep=";",index=False,header=False)

def write_singel_string_wares_attribute_to_csv_as_IBPmaterial(wares):
  IBPmaterial = pd.DataFrame(columns=['MANDT','LOGSYS','MATNR','LVORM','MATKL','MEINS','PRDHA','MTART','CUST_ATTR1','CUST_ATTR2','CUST_ATTR3','CUST_ATTR4','CUST_ATTR5','CUST_ATTR6','CUST_ATTR7','CUST_ATTR8','CUST_ATTR9','CUST_ATTR10'])

  wares = re.sub(r".*wares\:", "", wares)
  Dict = json.loads(wares)
  wares_amount_data = pd.DataFrame.from_dict(Dict.items())

  wares_amount_data.columns = ['ware', 'amount']
  IBPmaterial["MATNR"]=wares_amount_data["ware"]
  IBPmaterial["MEINS"]="piece"
  IBPmaterial["MANDT"]="001"
  IBPmaterial["LOGSYS"]="WDL"
  IBpmaterialdes = pd.DataFrame(columns=["MANDT","LOGSYS","RESID","SPRAS","KTEXT"])
  IBpmaterialdes["RESID"]=IBPmaterial["MATNR"]
  IBpmaterialdes["SPRAS"]="E"
  IBpmaterialdes["MANDT"]="001"
  IBpmaterialdes["LOGSYS"]="WDL"
  IBpmaterialdes.to_csv("materialsdesc.csv",sep=";",index=False,header=False)
  IBPmaterial.to_csv("materials.csv",sep=";",index=False,header=False)



def write_one_buildings_entry_to_IBPstock(building_entry,filename):
  IBPstocks = pd.DataFrame(columns=['MANDT','DELKZ','LOGSYS','MATNR','LOC_WERKS','LGORT','PLAAB','BLANR','LOC_LIFNR','LOC_KUNNR','CHARG','TST01','VRFKZ','MNG01'])
  wares=building_entry["wares"]
  wares = re.sub(r".*wares\:", "", wares)
  Dict = json.loads(wares)
  wares_amount_data = pd.DataFrame.from_dict(Dict.items())
  wares_amount_data.columns = ['ware', 'amount']
  IBPstocks["MATNR"]=wares_amount_data["ware"]
  IBPstocks["MNG01"]=wares_amount_data["amount"]
  IBPstocks['LOC_WERKS']=str(building_entry["serial"])
  IBPstocks["VRFKZ"]="X"
  IBPstocks['MANDT']="001"
  IBPstocks['LOGSYS']="WDL"
  IBPstocks.to_csv(filename+"stocks.csv",sep=";",index=False,header=False)

def write_one_buildings_orders_to_IBPorders(orders):
  IBPorders = pd.DataFrame(columns=['MANDT','LOGSYS','DELNR','DELPS','DELET','DELKZ','MATNR','LOC_WERKS',
  'CHARG','PLAAB','PLANR','LGORT','LOC_LIFNR','LOC_KUNNR','TST01','VRFKZ','MNG01','MNG02',
  'MNG03','TST02','FIX01','LOC_WERKS_FROM','VERID','DELVR','AUFVR','POSVR',
  'INFNR','EKORG','ESOKZ','MNG04','DEPLOYMENT','MOT_ID','EINVR','DPS_TST','PP_STAGE','REL_FIXED_STATUS'])
  IBPorders['MANDT']="001"
  IBPorders['LOGSYS']="WDL"
  IBPorders['DELNR']=orders.serial.apply(str)+orders.name.apply(str)+orders.time.apply(str)+orders.requested_Ware.apply(str)+orders.amount.apply(str)
  IBPorders.to_csv("IBPorders.csv")

  




  


In [2]:
data = read_data_from_files(directory)


In [3]:
a=map_to_real_time(read_data_from_files(directory))


In [4]:
string=a[0].loc[(a[0]['name'] == "atlanteans_headquarters") ]["wares"][0]
write_singel_string_wares_attribute_to_csv_as_IBPmaterial(a[0].loc[(a[0]['name'] == "atlanteans_headquarters") ]["wares"][0])

In [5]:
write_one_buildings_entry_to_IBPstock(a[0].loc[(a[0]['name'] == "atlanteans_headquarters") ].iloc[0],str(a[0].loc[(a[0]['name'] == "atlanteans_headquarters") ].iloc[0]["serial"]))

In [6]:
string=a[0].loc[(a[0]['name'] == "atlanteans_headquarters") ]["wares"][0]
string = re.sub(r".*wares\:", "", string)

In [7]:
Dict = json.loads(string)
fuck = pd.DataFrame.from_dict(Dict.items())
fuck.columns = ['ware', 'amount']

In [8]:
orders=read_orders_data_from_files(directory)
print((orders[0]["time"]))
orders_maped = map_to_real_time(orders)

0          18763
1          18763
2          18763
3          18763
4          18963
          ...   
7667    14381283
7668    14381444
7669    14381721
7670    14383454
7671    14383654
Name: time, Length: 7672, dtype: int64


In [9]:
#print((orders[0]["time"]))
print(orders_maped[0].columns)

Index(['serial', 'name', 'time', 'requested_Ware', 'amount'], dtype='object')


In [10]:

orders[0].rename(columns={'building_serial':'serial'}, inplace=True)
merged_data = a[0].merge(orders[0], on=["serial","time"], how='left')


In [11]:
print(merged_data[~merged_data["requested_Ware"].isnull()].columns)

Index(['serial', 'name_x', 'time', 'ware_economy_serial',
       'worker_economy_serial', 'position', 'reserved_by_worker', 'workers',
       'is_mine', 'is_port', 'needs_seafaring', 'needs_water_ways', 'passable',
       'is_production_site', 'is_warehouse', 'is_market', 'ware_priorities',
       'wares', 'input_queues', 'produced_Wares', 'workinpositions', 'name_y',
       'requested_Ware', 'amount'],
      dtype='object')


In [12]:
for i in range(len(data)):
  write_series_of_location_to_csv_as_IBPlocation(data[i][["serial","name"]].drop_duplicates(subset="serial"),str(i))




In [13]:
a=data[i][["serial","name"]].drop_duplicates(subset="serial")

In [14]:
write_one_buildings_orders_to_IBPorders(orders[0])